In [14]:
# import relevant modules 
import os
import sys
from importlib import reload
import numpy as np
from sympy import Matrix
from math import isclose
import matplotlib.pyplot as plt

# locate directory containing scripts 
current_directory = os.getcwd() 
src_directory = os.path.dirname(current_directory) + '/src'
sys.path.append(src_directory)

# import custom modules
import spin_chain, blue_red_bonds, generate_VTA_list, VTA_numerics, VTA_analytics, simulate_algorithm
reload(spin_chain)
reload(blue_red_bonds)
reload(generate_VTA_list)
reload(VTA_numerics)
reload(VTA_analytics)
reload(simulate_algorithm); 

In [3]:
# specify parameters for spin chain (number of sites, coupling constants, 
# periodic or nonperiodic boundary conditions,  and tolerance 
# with which to determine orthonormality of eigenstates
N = 6
Jx = 1
Jy = 1
Jz = 1
periodic_bc = True
tolerance = 1e-12

# collect properties of spin chain
H6, H6_list, eigenstates6, eigenvalues6, E0, ρ_ground_state6 = \
    spin_chain.properties(N, Jx, Jy, Jz, periodic_bc, tolerance)

In [4]:
sp_H6 = Matrix(H6.full())

In [5]:
spectrum_list = [-2*np.sqrt(13) - 4] + \
                [-2*np.sqrt(5) - 4]*3 + \
                [-6] + [-np.sqrt(17) - 1]*6 + \
                [-4]*6 + [-2]*7 + [0]*10 + \
                [-4 + 2*np.sqrt(5)]*3 + [2]*3 + \
                [-1 + np.sqrt(17)]*6 + \
                [-4 + 2*np.sqrt(13)] + \
                [4]*10 + [6]*7

In [6]:
λ_bool_list = []
for λ1, λ2 in zip(eigenvalues6, spectrum_list): 
    λ_bool_list.append(isclose(λ1, λ2, abs_tol = 1e-3))
all(λ_bool_list)

True

In [7]:
set(spectrum_list)

{-11.21110255092798,
 -8.47213595499958,
 -6,
 -5.123105625617661,
 -4,
 -2,
 0,
 0.4721359549995796,
 2,
 3.1231056256176606,
 3.2111025509279782,
 4,
 6}

In [8]:
H6

Quantum object: dims=[[2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2]], shape=(64, 64), type='oper', dtype=CSR, isherm=True
Qobj data =
[[6. 0. 0. ... 0. 0. 0.]
 [0. 2. 2. ... 0. 0. 0.]
 [0. 2. 2. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 2. 2. 0.]
 [0. 0. 0. ... 2. 2. 0.]
 [0. 0. 0. ... 0. 0. 6.]]

In [9]:
# compute simulatenous eigenvectors H4, S2, Sz, and σ24
H6, S2, Sz, σ36 = spin_chain.collect_spin_ops(N, Jx, Jy, Jz, periodic_bc)

In [12]:
(2**18)*100

26214400

In [13]:
412+61+8

481

In [29]:
from qutip import qzero, qeye, sigmax, sigmay, sigmaz, tensor

In [20]:
def generate_SWAP_operators(N, Jx, Jy, Jz):
    
    '''
    generate list of SWAP operators that needed to compute VTA 
    using the SU(2) expansion method
    '''
    
    if N % 2 != 0: 
        raise ValueError("Please enter an even number of sites.")
        
    # define zero and identity matrices corresponding to dimensions of VTA
    zeros_N = qzero([2]*N)
    I_N = qeye([2]*N)
    
    # define Pauli matrices and constants
    σ_x = sigmax()
    σ_y = sigmay()
    σ_z = sigmaz()
    
    # Interaction coefficients, which we assume are uniform throughout the lattice
    Jx_list = Jx*np.ones(N)
    Jy_list = Jy*np.ones(N)
    Jz_list = Jz*np.ones(N)

    # Setup operators for individual qubits; 
    # here σ_x_list[j] = X_j, σ_y_list[j] = Y_j, and σ_z_list[j] = Z_j
    # since the Pauli matrix occupies the jth location in the tensor product of N terms
    # for which (N-1) terms are the identity
    σ_x_list, σ_y_list, σ_z_list = [], [], []

    for i in range(N):
        op_list = [qeye(2)]*N
        op_list[i] = σ_x
        σ_x_list.append(tensor(op_list))
        op_list[i] = σ_y
        σ_y_list.append(tensor(op_list))
        op_list[i] = σ_z
        σ_z_list.append(tensor(op_list))

    # define empty lists for + and - projection operators
    π_list = []
    
    # collect list of all tuples corresponding to π_p and π_m 
    # SWAP operators
    for k in range(N):

        # find H_ij, the Hamiltonian between the ith and jth sites 
        H_kl = Jx_list[k] * σ_x_list[k] * σ_x_list[(k + 1) % N] + \
               Jy_list[k] * σ_y_list[k] * σ_y_list[(k + 1) % N] + \
               Jz_list[k] * σ_z_list[k] * σ_z_list[(k + 1) % N]
        
        # add π_p to π_m to π_p_list and π_m_list, respectively
        π_p = (3 + H_kl)/4
        π_m = (1 - H_kl)/4
        π_list.append((π_p, π_m))
    
    # check to ensure projectors obey established summation and orthogonality relations
    π_kl_bool_list = []
    for π_kl in π_list: 
        π_kl_bool_list.append(π_kl[0] * π_kl[1] == zeros_N and \
                              π_kl[0] + π_kl[1] == I_N)

    if all(π_kl_bool_list):
#         display(Latex(r'$ \pi^{+}_{kl} \pi^{-}_{kl} = 0 \text{ and } $'
#                       r'$\pi^{+}_{kl} + \pi^{-}_{kl} = \mathbb{1}$' 
#                      rf'$ \ \forall \ k,l \in \{{1, \dots, {N} \}}$'))
        return π_list
    else: 
        display(Latex(r'$ \pi^{+}_{kl} \pi^{-}_{kl} \neq 0 \text{ of } $'
                      r'$\pi^{+}_{kl} + \pi^{-}_{kl} \neq \mathbb{1}$' 
                     rf'$ \ \forall \ k,l \in \{{1, \dots, {N} \}}$'))
        raise ValueError(f'SWAP operators do not obey the desired summation and' + \
                          ' orthogonality conditions')

In [21]:
def MPO(π_list, α, E_0, b7, b6, b5, b4, b3, b2, b1, b0):
    
    '''
    generate matrix product operator corresponding to a unique α 
    and set of eight indices
    '''
    
    # define projection operators where π_kl is a tuple 
    # such that π_kl[0] = π^{+}_{kl} and π_kl[1] = π^{-}_{kl}
    π12 = π_list[0]
    π23 = π_list[1]
    π34 = π_list[2]
    π41 = π_list[3]

    # define constant q
    q = 2 + E_0/2
    
    # return matrix product operator for a given α and set of indices
    return np.exp((-2*α**2) * (
           (q - 4 + 2*(b0 + b3 + b5 + b6))**2 + \
           (q - 4 + 2*(b0 + b2 + b5 + b7))**2 + \
           (q - 4 + 2*(b1 + b2 + b4 + b7))**2 + \
           (q - 4 + 2*(b1 + b3 + b4 + b6))**2)) * \
            π41[b7]*π23[b6]*π34[b5]*π12[b4] * \
            π41[b3]*π23[b2]*π34[b1]*π12[b0]

In [22]:
def SU2_automated(N, α_start, α_end, α_steps, Jx, Jy, Jz, E_0): 
    
    '''
    compute list of VTAs using an efficient, automated approach
    '''
    
    # define array over which we will sweep α
    α_array = np.linspace(α_start, α_end, α_steps)
    
    # collect list of SWAP operators
    π_list = generate_SWAP_operators(N, Jx, Jy, Jz)

    # generate all possible combinations of tuples 
    combination_tuples = list(product([0, 1], repeat=int(N*(N/2))))
    
    # return the sum of all VTA
    return [sum(MPO(π_list, α, E_0, *combination) \
                for combination in combination_tuples) \
                for α in α_array]

In [31]:
N = 4
π_list = generate_SWAP_operators(N, Jx, Jy, Jz)

In [36]:
α = 1
MPO(π_list, α, E_0, b7, b6, b5, b4, b3, b2, b1, b0)

NameError: name 'E_0' is not defined